### Configuración Inicial

In [1]:
!pip uninstall -y numpy
!pip install numpy==1.26

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 54.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.0 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.0 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>

In [1]:
!pip install scikit-surprise --no-build-isolation --no-deps
!pip install memory_profiler

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2708550 sha256=fca78213029c74c0c11029ca2b278fe340df895dc234f3f7462bb3b56f64b904
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


### Instalación de Librerías

In [2]:
import pandas as pd
import gdown
from surprise import Dataset, Reader, SVDpp, accuracy
from surprise.model_selection import train_test_split
import time
from memory_profiler import memory_usage
from collections import defaultdict

### Importación de los Datos

In [3]:
gdown.download(id='1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v', output='training_ratings.csv', quiet=False)
gdown.download(id='1pKmf07ehHOmlvIyT8nv__vPuWE2Z3ygZ', output='validation_ratings.csv', quiet=False)

df_train = pd.read_csv('training_ratings.csv')
df_val = pd.read_csv('validation_ratings.csv')

Downloading...
From (original): https://drive.google.com/uc?id=1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v
From (redirected): https://drive.google.com/uc?id=1H_24ycns6zbOVfHFJRI9vGjVffVA5z6v&confirm=t&uuid=bd889cad-f63b-43b0-bf37-7c59ea5965ad
To: /content/training_ratings.csv
100%|██████████| 249M/249M [00:02<00:00, 85.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pKmf07ehHOmlvIyT8nv__vPuWE2Z3ygZ
To: /content/validation_ratings.csv
100%|██████████| 58.3M/58.3M [00:00<00:00, 96.7MB/s]


### Preprocesamiento de Datos

In [5]:
df_train.drop_duplicates(inplace=True, subset=['user', 'item'])
df_val.drop_duplicates(inplace=True, subset=['user', 'item'])

In [6]:
print(f"Tamaño original del training set: {len(df_train)}")

# se obtiene un sample debido a que hay muchos datos y se demora mucho
df_train_sample = df_train.sample(n=1000000, random_state=42)
print(f"Tamaño del nuevo training set (muestra): {len(df_train_sample)}")


Tamaño original del training set: 12390406
Tamaño del nuevo training set (muestra): 1000000


### Configuración de Experimentos

In [7]:
reader = Reader(rating_scale=(0, 10))
train_data = Dataset.load_from_df(df_train_sample[['user', 'item', 'rating']], reader)
trainset = train_data.build_full_trainset()

validation_tuples = [tuple(x) for x in df_val[['user', 'item', 'rating']].to_numpy()]

### Predicción de ratings y top N



In [ ]:
def evaluar_svdpp(trainset, testset, n_factors, n_epochs, lr_all, reg_all, verbose):
    """
    Entrena, predice y evalúa el modelo SVD++, retornando el RMSE y MAE.
    """
    algo = SVDpp(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all, verbose=verbose)

    algo.fit(trainset)

    predictions = algo.test(testset)

    rmse = accuracy.rmse(predictions)
    mae = accuracy.mae(predictions)

    return rmse, mae, predictions

start_time = time.time()

rmse_resultado, mae_resultado, predictions = evaluar_svdpp(trainset, validation_tuples, 50, 20, 0.01, 0.02, True)

end_time = time.time()
elapsed_time = end_time - start_time

memoria_usada = memory_usage((evaluar_svdpp, (trainset, validation_tuples, 50, 20, 0.01, 0.02, False)))
memoria_max = max(memoria_usada) - min(memoria_usada)

print(f"\n--- Resumen de Rendimiento (SVD++) ---")
print(f"RMSE final: {rmse_resultado:.4f}")
print(f"MAE final: {mae_resultado:.4f}")
print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")
print(f"Memoria utilizada: {memoria_max:.2f} MB")

 processing epoch 0
 processing epoch 1
 processing epoch 2
 processing epoch 3
 processing epoch 4
 processing epoch 5
 processing epoch 6
 processing epoch 7
 processing epoch 8
 processing epoch 9
 processing epoch 10
 processing epoch 11
 processing epoch 12
 processing epoch 13
 processing epoch 14
 processing epoch 15
 processing epoch 16
 processing epoch 17
 processing epoch 18
 processing epoch 19
RMSE: 1.3126
MAE:  0.9976
RMSE: 1.3119
MAE:  0.9972

--- Resumen de Rendimiento (SVD++) ---
RMSE final: 1.3126
MAE final: 0.9976
Tiempo de ejecución: 194.92 segundos
Memoria utilizada: 1409.13 MB


In [ ]:
def get_top_n(predictions, n=10):
    """Devuelve las N-mejores recomendaciones para cada usuario."""
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n_predictions = get_top_n(predictions, n=10)

user_id_ejemplo = df_val['user'].iloc[0]
if user_id_ejemplo in top_n_predictions:
    print(f"\nTop 10 predicciones para el usuario '{user_id_ejemplo}':")
    for iid, est_rating in top_n_predictions[user_id_ejemplo]:
        print(f"  Ítem ID: {iid}, Rating Predicho: {est_rating:.2f}")


Top 10 predicciones para el usuario ' beastvol':
  Ítem ID: 18602, Rating Predicho: 7.34


Grupales:

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score

print("Convirtiendo predicciones de Surprise a DataFrame...")
preds_df = pd.DataFrame(predictions, columns=['userID', 'itemID', 'rating', 'score', 'details'])

# Un rating >= 7 se considera una interacción positiva (relevante).
preds_df['label'] = (preds_df['rating'] >= 7).astype(int)

df_eval = preds_df[['userID', 'itemID', 'label', 'score']]
print("DataFrame de evaluación creado con éxito.")
print(df_eval.head())



print("\nCreando grupos sintéticos...")
user_counts = df_eval['userID'].value_counts()
# Nos quedamos con usuarios que tienen al menos 10 ratings para formar grupos más consistentes
valid_users = user_counts[user_counts >= 10].index.tolist()

np.random.seed(42)
num_groups = 1000
group_size = 4
groups = [np.random.choice(valid_users, group_size, replace=False) for _ in range(num_groups)]

print(f"Se crearon {len(groups)} grupos sintéticos de tamaño {group_size}.")
print("Ejemplo de un grupo:", groups[0])


# --- PASO 3: Agregar Predicciones para cada Grupo ---

print("\nAgregando predicciones para cada grupo...")
all_group_recs = []

for group_id, user_ids in enumerate(groups):
    group_predictions = df_eval[df_eval['userID'].isin(user_ids)]

    item_scores_per_group = group_predictions.groupby('itemID').agg(
        # Estrategias de agregación
        avg_score=('score', 'mean'),
        min_score=('score', 'min'),
        max_score=('score', 'max'),

        # Ground Truth del grupo: Un ítem es relevante si AL MENOS UN miembro le dio un rating positivo.
        group_label=('label', lambda x: 1 if x.sum() > 0 else 0)
    ).reset_index()

    item_scores_per_group['group_id'] = group_id
    all_group_recs.append(item_scores_per_group)

df_group_eval = pd.concat(all_group_recs, ignore_index=True)
print("Agregación completada.")
print(df_group_eval.head())


# --- PASO 4: Funciones de Métrica y Evaluación Final ---

def precision_recall_at_k(group, k):
    """Calcula Precision@K y Recall@K para un solo grupo."""
    group = group.sort_values('score', ascending=False)
    topk = group.head(k)
    hits = topk['label'].sum()
    total_relevant = group['label'].sum()
    precision = hits / k if k > 0 else 0
    recall = hits / total_relevant if total_relevant > 0 else 0
    return precision, recall

def ndcg_at_k(group, k):
    """Calcula nDCG@K para un solo grupo."""
    if group['label'].sum() == 0:
        return 0.0
    ranked_group = group.sort_values('score', ascending=False).head(k)
    if len(ranked_group) < 2:
        return 0.0
    true_relevance = np.asarray([ranked_group['label'].values])
    predicted_scores = np.asarray([ranked_group['score'].values])
    return ndcg_score(true_relevance, predicted_scores)

strategies = {
    'Average': 'avg_score',
    'Least Misery': 'min_score',
    'Most Pleasure': 'max_score'
}

group_results = []
K_values = [10]
for strategy_name, score_column in strategies.items():
    print(f"\nEvaluando estrategia: {strategy_name}...")
    df_strategy_eval = df_group_eval[['group_id', 'itemID', 'group_label']].copy()
    df_strategy_eval.rename(columns={'group_label': 'label'}, inplace=True)
    df_strategy_eval['score'] = df_group_eval[score_column]

    grouped_strategy = df_strategy_eval.groupby('group_id')

    for k in K_values:
        metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
        avg_precision = np.mean([m[0] for m in metrics])
        avg_recall = np.mean([m[1] for m in metrics])
        ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))
        avg_ndcg = np.mean(ndcg_scores)
        group_results.append({
            'Strategy': strategy_name,
            'K': k,
            'Precision@K': avg_precision,
            'Recall@K': avg_recall,
            'nDCG@K': avg_ndcg
        })

group_results_df = pd.DataFrame(group_results)

print("\n--- Resultados de Evaluación Grupal para SVD++ ---")
print(group_results_df)

Convirtiendo predicciones de Surprise a DataFrame...
DataFrame de evaluación creado con éxito.
      userID  itemID  label     score
0   beastvol   18602      1  7.745524
1    mycroft     278      0  7.194656
2    mycroft       5      1  8.507310
3   -=Yod@=-  264295      1  7.253843
4   -=Yod@=-   38949      0  6.351163

Creando grupos sintéticos...
Se crearon 1000 grupos sintéticos de tamaño 4.
Ejemplo de un grupo: ['Sidewynnder' 'razorclovis' 'dougielb' 'sandro']

Agregando predicciones para cada grupo...
Agregación completada.
   itemID  avg_score  min_score  max_score  group_label  group_id
0      50   7.026009   7.026009   7.026009            1         0
1      71   7.002014   7.002014   7.002014            1         0
2     171   7.166400   7.166400   7.166400            1         0
3     258   6.860341   6.860341   6.860341            1         0
4     438   6.674439   6.674439   6.674439            1         0

Evaluando estrategia: Average...


/tmp/ipython-input-3191658433.py:96: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-3191658433.py:99: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))



Evaluando estrategia: Least Misery...


/tmp/ipython-input-3191658433.py:96: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))
/tmp/ipython-input-3191658433.py:99: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))



Evaluando estrategia: Most Pleasure...


/tmp/ipython-input-3191658433.py:96: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics = grouped_strategy.apply(lambda x: precision_recall_at_k(x, k))



--- Resultados de Evaluación Grupal para SVD++ ---
        Strategy   K  Precision@K  Recall@K    nDCG@K
0        Average  10       0.9039  0.142668  0.965189
1   Least Misery  10       0.9033  0.142596  0.965215
2  Most Pleasure  10       0.9065  0.143025  0.967000


/tmp/ipython-input-3191658433.py:99: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = grouped_strategy.apply(lambda x: ndcg_at_k(x, k))


Individuales:

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score

print("Convirtiendo predicciones de Surprise a DataFrame...")
preds_df = pd.DataFrame(predictions, columns=['userID', 'itemID', 'rating', 'score', 'details'])

# Un rating >= 7 se considera una interacción positiva (relevante).
preds_df['label'] = (preds_df['rating'] >= 7).astype(int)

df_eval = preds_df[['userID', 'itemID', 'label', 'score']]
print("DataFrame de evaluación creado con éxito.")
print(df_eval.head())



K = 10

# df_eval: userID, itemID, label (0/1), score (predicción SVD++)
user_groups = df_eval.groupby('userID')

precisions = []
recalls = []
ndcgs = []

for uid, group in user_groups:
    # Si el usuario no tiene ningún ítem relevante, lo saltamos
    if group['label'].sum() == 0:
        continue

    # Ordenamos por score (predicción) de mayor a menor
    group_sorted = group.sort_values('score', ascending=False)

    # Top-K ítems recomendados para este usuario
    topk = group_sorted.head(K)

    hits = topk['label'].sum()
    total_relevant = group['label'].sum()

    precision = hits / K
    recall = hits / total_relevant if total_relevant > 0 else 0.0

    # nDCG@K para este usuario
    true_rel = np.asarray([topk['label'].values])   # relevancia real 0/1
    scores = np.asarray([topk['score'].values])     # score predicho
    ndcg = ndcg_score(true_rel, scores)

    precisions.append(precision)
    recalls.append(recall)
    ndcgs.append(ndcg)

print(f"\n--- Métricas individuales SVD++ (K={K}) ---")
print(f"Precision@{K} promedio: {np.mean(precisions):.4f}")
print(f"Recall@{K} promedio:    {np.mean(recalls):.4f}")
print(f"nDCG@{K} promedio:       {np.mean(ndcgs):.4f}")


Convirtiendo predicciones de Surprise a DataFrame...
DataFrame de evaluación creado con éxito.
      userID  itemID  label     score
0   beastvol   18602      1  7.341554
1    mycroft     278      0  7.166028
2    mycroft       5      1  8.502313
3   -=Yod@=-  264295      1  7.413006
4   -=Yod@=-   38949      0  6.566006


ValueError: Computing NDCG is only meaningful when there is more than 1 document. Got 1 instead.